En esta celda definimos los objetos curva de Tipos de Interés.


1.   **Curva estática ( IRCurve)**: Ésta curva es independiente de modelo y sería la que Bootstrapearíamos de precios de IRS. Es una interpolación en splines de logaritmo de Factores de Descuento.

2.   **Curva del LGM (LGMCurve) **: Se trata de una curva dinámica (de acuerdo al LGM). En un instante futuro, esta curva depende de la curva estática y del valor que pudiera tomar la variable x.


# **Curva LGM**

Sea 

$$
dx_t = \sigma dW^{\mathcal{N}}_t \;\;\;\; s.t \;\;\;\; x(0) = 0
$$

la dinámica de la variable que establece el comportamiento de la curva de tipos de interés.

El modelo LGM, establece la relación que existe entre esta variable y el numerario a una fecha futura.

$$
N(t, x_t) = \frac{1}{B(0, t)} exp \bigg( -H_t x_t - \frac{1}{2} H_t^2 \zeta_t \bigg)
$$

donde $B(0, t)$ es el factor de descuento a vencimiento $t$ con la curva estática. $H_t = \frac{1-e^{-\kappa t}}{\kappa}$

En clase demostramos que, dado el numeraio, el factor de descuento (según LGM) a una feha futura, $B(t, T, x_t)$ , como función de $x_t$ venía dado como,

$$
B(t, T, x_t) = \frac{B(0, T)}{B(0, t)} exp\bigg( - \left( H_T - H_t \right) x_t - \frac{1}{2} \left( H_T^2 - H_t^2\right) \zeta_t \bigg)
$$


$ \textbf{Pregunta 1: Demostrar la ecuación de arriba.}$

$ \textbf{Pregunta 2: Explica cómo impacta  $H_t$ o $\kappa$ en la forma de la curva.}$


In [0]:

##########################################################
#
#     NO ES NECESARIO TOCAR ESTA FUNCIÓN
#
##########################################################


import numpy as np
from scipy.interpolate import interp1d
import scipy as sc
import matplotlib.pyplot as plt



plt.style.use('default')

# ------------------------------------------------

# Clase Curva
class IRCurve:
  
  # CONSTRUCTOR (Da vida a la clase)
  def __init__(self, dates, dfs):
    self._ref_date = dates[0]
    self._dfs = dfs
    self._times = (dates - dates[0]) / 365.25
    self._rates_time = -np.log(dfs)
    self._curve = interp1d(self._times, self._rates_time, kind = 'cubic')
  
  # Devuelve B(t, T) = B(0, T) / B(0, t)
  def get_zero(self, t, T):
    B_0_T = np.exp(-self._curve((T-self._ref_date)/365.25))
    B_0_t = np.exp(-self._curve((t-self._ref_date)/365.25))
    
    return B_0_T / B_0_t
  
 # -----------------------------------------------------
 
def H(kappa, time):
  return (1 - np.exp(-kappa * time)) / kappa
  
  
# Definición de la clase LGM
class LGMCurve:

  # CONSTRUCTOR
  def __init__(self, IR_curve, kappa, sigma):
    self._curve = IR_curve
    self._kappa = kappa
    self._sigma = sigma

  # Numerario: N(t, xt)
  def numeraire(self, t, xt):

    time_to_t = (t - self._curve._ref_date) / 365.25
    ht = H(self._kappa, time_to_t)
    zeta_t = (self._sigma ** 2) * time_to_t

    exponent = ht * xt + 0.5 * (ht**2) * zeta_t

    return 1 / self._curve.get_zero(self._curve._ref_date, t) \
            * np.exp(exponent)
  
  # B(t, T, xt)
  def get_zero_x(self, t, T, xt):

    time_to_t = (t - self._curve._ref_date) / 365.25
    time_to_T = (T - self._curve._ref_date) / 365.25

    ht = H(self._kappa, time_to_t)
    hT = H(self._kappa, time_to_T)

    zeta = (self._sigma * self._sigma) * time_to_t

    df = self._curve.get_zero(t, T) 
    exponent = (hT -ht ) * xt + 0.5 * (hT * hT - ht * ht) * zeta 

    df *= np.exp(-exponent)

    return df
    
    
 # -----------------------------------------------------

**En la siguiente celda, inicializamos la fecha valor (today) y la curva estática (ir_curve). A lo largo de todo el ejercicio utilizaremos la misma curva**



In [0]:



#################################################################################

today = 43561 # 6 de Abril de 2019
last_date = today + 365.25 * 40

# Defino las fechas pilar para los factores de desceunto ...
pillar_dates = np.linspace(today, last_date, 50)

# Defino los rates como curva no plana
dates_rates = np.array([today, today + 365, today + 365 * 5, today + 365 * 10, \
                       today +365 * 20, today + 365 * 30, today + 365 * 50])
rates = np.array([0.0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.028])

# Calculo los FD de acuerdo a esa estructura no plana de tipos
dfs = np.zeros(len(pillar_dates))
rate_interp = interp1d(dates_rates, rates, kind = 'linear')
for i in range( len(pillar_dates)):
  rate = rate_interp(pillar_dates[i])
  dfs[i] = (np.exp(-rate * (pillar_dates[i] - today)/ 365.25))

# Definimos IR Curve (Curva original)  
ir_curve = IRCurve(pillar_dates, dfs)

#################################################################################






# **Función de valoración de un Swaption de acuerdo al LGM**

Supongamos un Call-Money-Swap cuya fecha de comienzo denotamos por $T_a$ y última fecha de pago es $T_b$. Supongamos una opción que nos da derecho a entrar en ese IRS en la fecha $T_a$. La prima forward de este swaption (de acuerdo al LGM) puede expresarse como:

\begin{equation}
\frac{V_t}{B(t, T_a)} = E^\mathcal{N}\left(  \left( 1- B(T_a, T_b, x(T_a)) - K \sum_{j=1}^N \Delta_j B(T_a, T_j, x(T_a)) \right)^+ \bigg| \mathcal{F}_t \right) 
\end{equation}

Vimos en clase, que como el valor de este IRS es monótono en $x(T_a)$, podemos buscar el valor $x^{*}$ para el cual, 

$$
1- B(T_a, T_b, x^*) - K \sum_{j=1}^N \Delta_j B(T_a, T_j, x^*) = 0
$$


Aplicando ésto, la prima forward puede expresarse como, 

\begin{equation}
\frac{V_t}{B(t, T_a)} = E^\mathcal{N}\left(  \left( B(T_a, T_b, x^*)- B(T_a, T_b, x(T_a)) \right) ^+ \bigg| \mathcal{F}_t \right) - K \sum_{j=1}^N \Delta_j  E^\mathcal{N}\left(\left( B(T_a, T_j, x^*) -  B(T_a, T_j, x(T_a)) \right)^+ \bigg| \mathcal{F}_t \right) 
\end{equation}



Finalmente, vimos en clase que la función de valoración podríamos expresarla en función de Black-Scholes como

\begin{equation}
\frac{V_t}{B(t, T_a)} = \sum_{j=1}^N  \omega_j BS^{LN}\bigg( \text{forward} =  \frac{B(t, T_j)}{B(t, T_a)},  \text{strike} = B(T_a, T_j,  x^*), \text{mty} = T_a, \text{varianza} = \sigma^2 \left( H_{T_a} - H_{T_j}\right)^2 \bigg) 
\end{equation}

donde 

$$
\omega_j=\left\{
                \begin{array}{ll}
                  K\Delta_j & \text{si} & j = 1, \ldots, N - 1\\
                  1 + K \Delta_N & text{si} & j = N\\
                \end{array}
              \right.
$$


# **EJERCICIO 1**

Calcular el precio de un swaption payer ATM ($T_a = \text{today} + 5Y$, $T_b = T_a + 10Y$) de acuerdo al LGM ($\sigma = 1\%$, $\kappa = 1 \%$), de acuerdo a los siguientes métodos:



1.   Función Analítica: Ésta es la que describimos más arriba, y cuya función ponemos en la siguiente celda.
2.   MC: Simula por MC la variable $x(T_a)$ y calcula 
$$
\frac{1}{B(t, T_a)}E_t \left( \frac{ IRS(T_a, x(T_a))^+}{N(T_a, x(Ta))} \bigg| \mathcal{F}_t \right)
$$

Calcula también el intervalo de confianza al 95%
3. Calcula el precio por integral numérica, 

$$
\frac{1}{B(t, T_a)} \int_{-\infty}^{\infty} \frac{ IRS(T_a, x)^+}{N(T_a, x)} \eta_{x(T_a)}(x) dx
$$

Tratad de relacionar (con vuestras palabras) el método 2 y 3.


4. Resuelve el precio por integral numérica donde la función de densidad la calcules derivando dos veces la función de valoración del swaption en (1) dos veces.

$$
V(t) = \int_{-\infty}^{\infty} \left(S -K \right)^{+} \frac{\partial^2 Swptn}{\partial K^2}\bigg|_{K = S}dS
$$

Donde $Swptn$ es la prima forward del apartado (1)
$$
Swptn = E^\mathcal{N}\left(  \left( 1- B(T_a, T_b, x(T_a)) - K \sum_{j=1}^N \Delta_j B(T_a, T_j, x(T_a)) \right)^+ \bigg| \mathcal{F}_t \right) 
$$

En la siguiente celda están definidos:

1. El instrumento IRS (IRS)
2. La función de valoración de un IRS de acuerdo al LGM (LGM_IRS_Price)
3. La función de valoración analítica de un swaption (LGM_Swaption_Price)

In [0]:

# CLASE IRS
class IRS:
  
  def __init__(self, start_date, tenor, freq, fixed_coupon):
    self.start_date = start_date
    self.end_date = start_date + tenor * 365.25
    self.fix_dates = np.linspace(self.start_date, self.end_date, tenor * freq) 
    self.fixed_coupon = fixed_coupon
    self.dcf = (self.fix_dates[1:] - self.fix_dates[0:-1])/360 # Act/360
    self.fix_dates = self.fix_dates[1:]

# --------------------------------------------------------
    
# Función que valora el IRS de acuerdo al LGM
def LGM_IRS_price(value_date, irs, lgm_curve, x_at_value_date):
  
  # Pata Flotante
  floating_leg = lgm_curve.get_zero_x(value_date, irs.start_date, x_at_value_date) - \
                 lgm_curve.get_zero_x(value_date, irs.end_date, x_at_value_date) 
    
  # Pata Fija
  fixed_leg = 0
  for i in range(len(irs.fix_dates)):
    fixed_leg += irs.dcf[i] * lgm_curve.get_zero_x(value_date, \
                                                   irs.fix_dates[i], x_at_value_date)
  
  fixed_leg *= irs.fixed_coupon
  
  return (floating_leg - fixed_leg)

# --------------------------------------------------------


# Función de valoración del swaption
def LGM_Swaption_Price(value_date, irs, lgm_curve):
  
  # 1. Calculamos x_star tal que IRS(T, x_star) = 0 
  T_mat = irs.start_date
  f_obj = lambda x : LGM_IRS_price(T_mat, irs, lgm_curve, x)
  x_star = optimize.newton(f_obj, 0)
  
  # Función anidada que calcula la put sobre ZC -------------
  def ZC_put(T_j):
    # Valoramos la última put sobre ZC
    fwd = lgm_curve.get_zero_x(value_date, T_j, 0) / \
          lgm_curve.get_zero_x(value_date, irs.start_date, 0)

    # strike B(T, T_N, x_star)
    strike = lgm_curve.get_zero_x(irs.start_date, T_j, x_star)

    # vol = (H_T - H_T_N) sigma_x
    vol = H(lgm_curve._kappa, (irs.start_date - value_date) / 365.25) - \
          H(lgm_curve._kappa, (T_j - value_date) / 365.25)
    
    vol *= lgm_curve._sigma
    
    vol = np.abs(vol)

    # Llamamos BS
    last_call = LN_BlackScholes(fwd, strike, \
                               (irs.start_date - value_date)/365.25, vol)
    last_put = last_call - (fwd - strike)
    
    return last_put
  # ------------------------------------------------------------
  
  # Iteramos por todas las fechas de pago del IRS y calculamos \
  # las puts sobre los ZC.
  
  swaption = 0 
  for i in range(len(irs.fix_dates)):
    # Calculate put
    put = ZC_put(irs.fix_dates[i])
    
    # Calculate K Delta o (1 + K Delta)
    notional = irs.dcf[i] * irs.fixed_coupon 
    if i == len(irs.fix_dates)-1:
      notional += 1 
      
      
    # Actualizamos el precio del swaption
    swaption += notional * put
  
  return swaption
  
  
# ---------------------------------------------------------------


# **EJERCICIO 2. **

Para el Swaption con las características del ejercicio 1,  ($T_a = \text{today} + 5Y$, $T_b = T_a + 10Y$) y  de acuerdo al LGM ($\sigma = 1\%$, $\kappa = 1 \%$),  calculad para los Strikes en ATM difference (ATM Differece = K - Forward) = $-200\;  bp, -100 \; bp, -50 \;  bp, 0 \; bp, 50 \; bp, 100 \; bp, 200 \; bp$ el precio del Swaption Payer Physical Settled vs Cash-Settled.

Recordad que el precio del Swaption Physical-Settled es el que habéis calculado en ejercicio 1. El Pay-Off a vencimiento del Csh-Settled es

$$
Swptn^\text{Cash}(T_a) = \left( S(T_a, x(T_a)) - K \right)^+ \sum_{j=1}^N \frac{ \Delta_j }{\left(1 + S(T_a, x(T_a)) \right)^{(T_j - T_a)}}
$$

Comparad ambas gráficas y tratad de justificar los resultados.

**Nota:** Podéis utilizar el método numérico que consideréis más oportuno. Espero algún comentario en relación a la elección del método.



# **EJERCICIO 3. **

Para el Swaption con las características del ejercicio 1,  ($T_a = \text{today} + 5Y$, $T_b = T_a + 10Y$) , asumamos que el precio del swaption payer de mercado viene establecido por el LGM ($\sigma = 1\%$, $\kappa = 1 \%$).


Para distintos valores de $\kappa = -5\%, -3\%, -1\%, 1\%, 3\%, 5\%$


1. Calculad el nuevo valor de $\sigma$ tal que el precio del swaption ATM es igual al original ($\sigma = 1\%$, $\kappa = 1 \%$).
2. Pintar el smile de volatilidad en términos normales para los strikes (en ATM difference) $-200\;  bp, -100 \; bp, -50 \;  bp, 0 \; bp, 50 \; bp, 100 \; bp, 200 \; bp$


In [0]:

from scipy.stats import norm

# Fórmula de BS en términos normales. La vais a necesitar para calcular el smile en términos normales.


# ----------------------------------------------------------------


# EVALUATE BS ..
def BS_norm_price(F, K, T, vol):
  '''
  Normal Black-Scholes
  '''
  sigma_sqrt_t = vol * np.sqrt(T)
  d1 = (F - K) / sigma_sqrt_t
  
  return (F-K) * norm.cdf(d1) + sigma_sqrt_t * norm.pdf(d1)
# ----------------------------------------------------------------

